---
<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>Comp 555 BioAlgorithms - Spring 2021</h1>
<h1 style="font-size: 250%;">Problem Set #4</h1>
<h1>Issued Tuesday, 4/1/2021; Due Tuesday, 4/15/2021</h1>
</div>

**Homework Information:** Some of the problems are probably too long to be done the night before the due date, so plan accordingly. Feel free to get help from others, but **the work you submit in should be your own.** Furthermore, this assignment will overlap Problem Set #5, so plan accordingly. 

**Warning:** This notebook has been annotated with metadata so that it can be uploaded to the grading system. It is very important that you enter your answers in the provided cells. You can add extra cells to explore approaches, but only the provided cell can and will be graded. Thus, if you delete a cell and add a replacement, there is a possiblity that your problem will not be graded. If you ever need to start over, you should download a new version of the problem set and transfer your solutions to it.

In [1]:
# Replace the following string values with the requested information
class Student:
    first = "Katelyn"
    last = "Cline"
    onyen = "katelyn9"
    pid = "730152239"

The following problems make use of the following Insulin protein sequences

In [2]:
human = "MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQGSLQPLALEGSLQKRGIVEQCCTSICSLYQLENYCN"
dog = "MALWMRLLPLLALLALWAPAPTRAFVNQHLCGSHLVEALYLVCGERGFFYTPKARREVEDLQVRDVELAGAPGEGGLQPLALEGALQKRGIVEQCCTSICSLYQLENYCN"

---
**Problem #1:** Find the longest common subsequence (LCS) shared between human and dog insulin, its length, and the alignment that it implies.

In [96]:
# from lecture 16 code
from numpy import *

def findLCS(v, w):
    score = zeros((len(v)+1,len(w)+1), dtype="int32")
    backt = zeros((len(v)+1,len(w)+1), dtype="int32")
    for i in range(1,len(v)+1):
        for j in range(1,len(w)+1):
            # find best score at each vertex
            if (v[i-1] == w[j-1]):  # test for a match ("diagonal street")
                score[i,j], backt[i,j] = max((score[i-1,j-1]+1,3), (score[i-1,j],1), (score[i,j-1],2))
            else:
                score[i,j], backt[i,j] = max((score[i-1,j],1), (score[i,j-1],2))
    return score, backt

def LCS(b,v,i,j):
    if ((i == 0) and (j == 0)):
        return ''
    elif (b[i,j] == 3):
        return LCS(b,v,i-1,j-1) + v[i-1]
    elif (b[i,j] == 2):
        return LCS(b,v,i,j-1)
    else:
        return LCS(b,v,i-1,j)
    
def Alignment(b,v,w,i,j):
    if ((i == 0) and (j == 0)):
        return ['','']
    if (b[i,j] == 3):
        result = Alignment(b,v,w,i-1,j-1)
        result[0] += v[i-1]
        result[1] += w[j-1]
        return result
    if (b[i,j] == 2):
        result = Alignment(b,v,w,i,j-1)
        result[0] += "_"
        result[1] += w[j-1]
        return result
    if (b[i,j] == 1):
        result = Alignment(b,v,w,i-1,j)
        result[0] += v[i-1]
        result[1] += "_"
        return result

In [97]:
s, b = findLCS(human, dog)
lcs = LCS(b,human,b.shape[0]-1,b.shape[1]-1)
print("LCS:", lcs)
print("Length of LCS:", len(lcs))
align = Alignment(b,human,dog,b.shape[0]-1,b.shape[1]-1)
lcsHuman = align[0]
lcsDog = align[1]
print("h =", align[0])
print("d =", align[1])

LCS: MALWMRLLPLLALLALWAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKRREEDLQGLQPLALEGLQKRGIVEQCCTSICSLYQLENYCN
Length of LCS: 90
h = MALWMRLLPLLALLALWGPDPA_A___AFVNQHLCGSHLVEALYLVCGERGFFYTPKT_RREA_EDLQ_______GS______LQPLALEGS_LQKRGIVEQCCTSICSLYQLENYCN
d = MALWMRLLPLLALLALW____APAPTRAFVNQHLCGSHLVEALYLVCGERGFFYTPK_ARRE_VEDLQVRDVELAG_APGEGGLQPLALEG_ALQKRGIVEQCCTSICSLYQLENYCN


---
**Problem #2:** Find the best *global alignment* between human and dog insulin assuming the following scoring from the "Alignment Game" described in Lecture 15, slide 4:

* +1 for a match
* 0 for a mismatch
* -1 for an indel

What is score for this alignment and the alignment itself? 

In [98]:
# from lecture 17
# need to either make an ugly score matrix with all the proteins where matches equal 1 and mismatches equal 0 and indel
    # equals -1
# OR actually understand code and matrix and set up score to do comparisons and add 1 if equal and subtract 1 if indel

import numpy

def GlobalAlign(v, w, indel):
    s = numpy.zeros((len(v)+1,len(w)+1), dtype="int32")
    b = numpy.zeros((len(v)+1,len(w)+1), dtype="int32")
    for i in range(0,len(v)+1):
        for j in range(0,len(w)+1):
            if (j == 0):
                if (i > 0):
                    s[i,j] = s[i-1,j] + indel
                    b[i,j] = 1
                continue
            if (i == 0):
                s[i,j] = s[i,j-1] + indel
                b[i,j] = 2
                continue
#            score = s[i-1,j-1] + scorematrix[v[i-1],w[j-1]]
            score = s[i-1,j-1]
            if(v[i-1] == w[j-1]):
                score += 1
            vskip = s[i-1,j] + indel
            wskip = s[i,j-1] + indel
            s[i,j] = max(vskip, wskip, score)
            if (s[i,j] == vskip):
                b[i,j] = 1
            elif (s[i,j] == wskip):
                b[i,j] = 2
            else:
                b[i,j] = 3
    return (s, b)

x, y = GlobalAlign(human,dog, -1)
print("Best score =", x[-1,-1])
align = Alignment(y,human,dog,y.shape[0]-1,y.shape[1]-1)
globalHuman = align[0]
globalDog = align[1]
print("human =", align[0])
print("dog =", align[1])

Best score = 78
human = MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQ_______GS_____LQPLALEGSLQKRGIVEQCCTSICSLYQLENYCN
dog = MALWMRLLPLLALLALWAPAPTRAFVNQHLCGSHLVEALYLVCGERGFFYTPKARREVEDLQVRDVELAGAPGEGGLQPLALEGALQKRGIVEQCCTSICSLYQLENYCN


**Problem #3:** According to the rules of the "Alignment Game", what score does your alignment from **Problem 1** achieve? How would the rules of the alignment game need to be changed so that your solution for **Problem 1** would be the optimal answer?

In [99]:
def AlignmentGame(h, d):
    score = 0
    for i in range(len(h)):
        if(h[i] == '_' or d[i] == '_'):
            #indel
            score += 0
        elif(h[i] == d[i]):
            #match
            score += 1
        else:
            #mismatch
            score += -1
    return score
lcsScore = AlignmentGame(lcsHuman, lcsDog)
globalScore = AlignmentGame(globalHuman, globalDog)


print("LCS: ",lcsScore)
print("Global: ", globalScore)


LCS:  90
Global:  82


---
**Problem #4:** Find the length of the longest *embedded* common subsequence in the **human** insulin sequence.

The longest embedded common subsequence is the longest subsequence that appears twice in a single given sequence for which no symbol matches itself. As an example, consider "TAGACTAAG". The longest common subsequence is the sequence itself, "TAGACTAAG", but the longest *embedded* common subsequence is "TAG". 

In [102]:
def modifyFindLCS(v, w):
    score = zeros((len(v)+1,len(w)+1), dtype="int32")
    backt = zeros((len(v)+1,len(w)+1), dtype="int32")
    for i in range(1,len(v)+1):
        for j in range(1,len(w)+1):
            # find best score at each vertex
            if (v[i-1] == w[j-1] and i != j):  # test for a match at different indices
                score[i,j], backt[i,j] = max((score[i-1,j-1]+1,3), (score[i-1,j],1), (score[i,j-1],2))
            else:
                score[i,j], backt[i,j] = max((score[i-1,j],1), (score[i,j-1],2))
    return score, backt

locations = []
def modifyLCS(b,v,i,j):
    if ((i == 0) and (j == 0)):
        # print('00')
        return ''
    elif (b[i,j] == 3):
        # print(j-1, i-1, v[i-1])
        locations.append((j-1, i-1, v[i-1]))
        return modifyLCS(b,v,i-1,j-1) + v[i-1]
    elif (b[i,j] == 2):
        # print('2')
        return modifyLCS(b,v,i,j-1)
    else:
        # print('0 or 1')
        return modifyLCS(b,v,i-1,j)


In [103]:
s, b = modifyFindLCS(human, human)
lcs = modifyLCS(b,human,b.shape[0]-1,b.shape[1]-1)
print("LCS:", lcs)
print(len(lcs))

LCS: MLLLLLALALGAVCGEGFTREAEGSLQESICSLYN
35


In [104]:
#test = "TAGACTAAG"
#s, b = modifyFindLCS(test, test)
#print(s)
#print(b)
#print(b.shape[0]-1)
#print(b.shape[1]-1)
#lcs = modifyLCS(b,test,b.shape[0]-1,b.shape[1]-1)
#print("LCS:", lcs)


**Problem #5:** Output the longest embedded common subsequence from the string given in **Problem #4** in the following comma separated format:

    positionInFirstSubsequence, positionInSecondSubsequence, commonSymbol
    
with one symbol per line. For "TAGACTAAG" the output could be:

    0,5,T
    1,7,A
    2,8,G


In [105]:
locations.reverse()
for x in locations:
    print(x)    

(0, 4, 'M')
(2, 7, 'L')
(6, 9, 'L')
(7, 10, 'L')
(9, 12, 'L')
(10, 13, 'L')
(11, 14, 'A')
(12, 15, 'L')
(14, 23, 'A')
(15, 29, 'L')
(17, 31, 'G')
(21, 37, 'A')
(25, 41, 'V')
(30, 42, 'C')
(31, 43, 'G')
(36, 44, 'E')
(43, 46, 'G')
(47, 48, 'F')
(50, 53, 'T')
(54, 55, 'R')
(56, 58, 'E')
(57, 68, 'A')
(58, 70, 'E')
(62, 71, 'G')
(63, 72, 'S')
(64, 73, 'L')
(65, 74, 'Q')
(70, 80, 'E')
(72, 85, 'S')
(78, 86, 'I')
(83, 87, 'C')
(85, 88, 'S')
(89, 92, 'L')
(90, 95, 'Y')
(94, 97, 'N')


---

## Instructions for submitting your problem set

When you are ready to submit a version of your problem set, follow the instructions below.

1. Press [Save and Checkpoint] on the *File* menu of your Jupyter notebook.
2. Press the link below, which will take you to a website for submitting your problem set.
3. Choose the ***correct problem set number*** from the pull-down, else you might overwrite an earlier submission.
4. Enter in your onyen and PID in the form provided, then upload your submission.

Click [here to submit](http://csbio.unc.edu/mcmillan/index.py?run=PS.upload) your completed problem set

**Instructions for resubmissions:**

1. You may resubmit as many times as you like before the deadline. 
2. Resubmissions *always* overwrite any earlier submissions. 
3. If you resubmit after the due date, you will be warned of any penalties. 
4. Problem sets will not be regraded.